### Description:  
This is the second step in the motif conservation analysis (results shown in shown in fig7).

In this script we look for motifs we have found in human herpes simplex virus (HSV1) within it's proteins' orthologs

### we perform a similar process in both dmi motifs and non-dmi motifs  
  
**reminder:**  
* "non-dmi" is a sequence that matches a regular expression (regex) pattern of a certain motif that was dropped for 1 or 2 reasons:
1. The motif does not match a domain of the human PPIs of said viral protein
2. The average plddt of said motif is higher than 50 

We get a table of occurrences - which tells us if the said motif was found in a certain protein's ortholog (through regular expression matching)

In [59]:
import os
import pandas as pd
import re

## HSV

In [143]:
# Function that removes "\n" from file from everyline besides the one after the sequence name in order to clean fasta files
def rreplace(s, old, new, occurrence):
    li = s.rsplit(old, occurrence)
    return new.join(li)

In [144]:
# Input files: output of script "01 - motif table dmi and not"
hsv_dmi_path = r"hsv_dmi_motifs.csv"
hsv_non_dmi_path = r"hsv_non_dmi_motifs.csv"

In [145]:
# Output path for dmi motifs
hsv_dmi_motifs_occurence_output=r"hsv_dmi_motifs_OCCURENCES.csv"

In [146]:
# Output path for non dmi motifs
hsv_non_dmi_motifs_occurence_output=r"hsv_non_dmi_motifs_OCCURENCES.csv"

### converting gene name to uniprot

In [148]:
hsv_uniprot_path= # path to hsv1_protein_level_table.csv 
hsv_uniprot_df = pd.read_csv(hsv_uniprot_path)

In [149]:
# Making a dictionary of uniprot ID : prot name
# Also making an inverted dictionary of  protein name : uniprot ID

hsv_uniprot_dict = dict(zip(hsv_uniprot_df["Uniprot ID"] , hsv_uniprot_df["Gene name"]))      
inverted_hsv_uniprot_dict = {value: key for key, value in hsv_uniprot_dict.items()}

### create orthologs list

In [151]:
hsv_sequences_path_in = r"MSA_DATA\MOTIF_SEARCH\HSV\MS"
hsv_proteins_list= os.listdir(hsv_sequences_path_in)

hsv_proteins_names_list = [file.split('.')[0] for file in hsv_proteins_list]

# Create list of uniprot IDs of the proteins with orthologs (instead of gene names)
unirpot_hsv_proteins_with_ortho = [inverted_hsv_uniprot_dict.get(i) for i in hsv_proteins_names_list ]

### list HSV1 dmi and non dmi motifs

In [154]:
hsv_dmi_df = pd.read_csv(hsv_dmi_path)
hsv_non_dmi_df = pd.read_csv(hsv_non_dmi_path)

In [156]:
hsv_dmi_proteins = list(hsv_dmi_df["v_prot"].unique())
hsv_non_dmi_proteins = list(hsv_non_dmi_df["v_prot"].unique())

### looking for HSV1 dmi motifs within orthologs

In [157]:
# Rows to append to in the following loop
rows_hsv_dmi_motifs_occurence_output = []

In [20]:
# Go over the list of dmi proteins
for protein in hsv_dmi_proteins:
    
    # Go on only if the protein has orthologs
    if protein in unirpot_hsv_proteins_with_ortho:
        print(protein)
        
        # Open the orthologs sequence file (with all the protein's orthologs sequences)
        gene_name = hsv_uniprot_dict.get(protein)
        with open(os.path.join(hsv_sequences_path_in , gene_name + ".fa" ), "r") as protein_sequences:

            # Read it
            file_as_string = protein_sequences.read()

            # Split
            seqs_list = file_as_string.split("*\n")
            seqs_list.pop()

            # Convert into dictionary. of variant and sequence.
            protein_variant_and_seqs_dict = {i[0].replace(",", "").replace(">", ""): i[1] for i in map(lambda i: rreplace(i, '\n', '', i.count('\n') - 1).split("\n"), seqs_list)}

            # Get sub dataframe of all the disordered motifs of that protein from the dmi table
            protein_dmi_df = hsv_dmi_df[hsv_dmi_df['v_prot'] == protein]

            # Make a list where each item is a sub list of [motif name, motif regex]
            protein_dmi_motifs = [(motif_name, motif_regex) for motif_name, motif_regex in zip(protein_dmi_df["motif_name"], protein_dmi_df["motif_regex"])]

            # Go over every motif of this protein and its regex and search for it in every variant's sequence (ortholog).
            for motif_name, regex in  protein_dmi_motifs:
                for variant, seq in protein_variant_and_seqs_dict.items():
                    matches = list(re.finditer(regex, seq))
                    if not matches:

                        hsv_dmi_motifs_occurence_row = {'viral_uniprot' : protein , 'viral_prot_name' : gene_name , 
                                                        'motif_name' : motif_name , 'variant': variant, 'ocurrence': 0 }

                        rows_hsv_dmi_motifs_occurence_output.append(hsv_dmi_motifs_occurence_row)

                    else:
                        hsv_dmi_motifs_occurence_row = {'viral_uniprot' : protein , 'viral_prot_name' : gene_name , 
                                                            'motif_name' : motif_name , 'variant': variant, 'ocurrence': 1 }

                        rows_hsv_dmi_motifs_occurence_output.append(hsv_dmi_motifs_occurence_row)
                            
                    


In [160]:
# Convert rows to tables
hsv_dmi_motifs_occurence_df = pd.DataFrame(rows_hsv_dmi_motifs_occurence_output)

In [161]:
# Save tables
hsv_dmi_motifs_occurence_df.to_csv(hsv_dmi_motifs_occurence_output , index= False)

### looking for HSV1 non-dmi motifs within orthologs

In [163]:
# Rows to append to in the following loop
rows_hsv_non_dmi_motifs_occurence_output = []

In [18]:
for protein in hsv_non_dmi_proteins:
    
    # Go on only if the protein has orthologs
    if protein in unirpot_hsv_proteins_with_ortho:
        print(protein)
        
        # Open the orthologs sequence file (with all the protein's orthologs sequences)
        gene_name = hsv_uniprot_dict.get(protein)
        with open(os.path.join(hsv_sequences_path_in , gene_name + ".fa" ), "r") as protein_sequences:

            # Read it
            file_as_string = protein_sequences.read()

            # Split
            seqs_list = file_as_string.split("*\n")
            seqs_list.pop()

            # Convert into dictionary of variant and sequence.
            protein_variant_and_seqs_dict = {i[0].replace(",", "").replace(">", ""): i[1] for i in map(lambda i: rreplace(i, '\n', '', i.count('\n') - 1).split("\n"), seqs_list)}

            # Get sub df of all the disordered motifs of that protein
            protein_non_dmi_df = hsv_non_dmi_df[hsv_non_dmi_df['v_prot'] == protein]

            # Make a list where each item is a sub list [motif name, motif regex]
            protein_non_dmi_motifs = [(motif_name, motif_regex) for motif_name, motif_regex in zip(protein_non_dmi_df["motif_name"], protein_non_dmi_df["motif_regex"])]

            # Go over every motif of this protein and its regex and search for it in every variant's sequence (ortholog).
            for motif_name, regex in  protein_non_dmi_motifs:
                for variant, seq in protein_variant_and_seqs_dict.items():
                    matches = list(re.finditer(regex, seq))
                    if not matches:

                        hsv_non_dmi_motifs_occurence_row = {'viral_uniprot' : protein , 'viral_prot_name' : gene_name , 
                                                        'motif_name' : motif_name , 'variant': variant, 'ocurrence': 0 }

                        rows_hsv_non_dmi_motifs_occurence_output.append(hsv_non_dmi_motifs_occurence_row)

                    else:
                        hsv_non_dmi_motifs_occurence_row = {'viral_uniprot' : protein , 'viral_prot_name' : gene_name , 
                                                            'motif_name' : motif_name , 'variant': variant, 'ocurrence': 1 }

                        rows_hsv_non_dmi_motifs_occurence_output.append(hsv_non_dmi_motifs_occurence_row)
                            
                    


In [16]:
# Convert rows to tables
hsv_non_dmi_motifs_occurence_df = pd.DataFrame(rows_hsv_non_dmi_motifs_occurence_output)

In [166]:
# Save tables
hsv_non_dmi_motifs_occurence_df.to_csv(hsv_non_dmi_motifs_occurence_output , index= False)
